In [68]:
import os
import numpy as np
from collections import OrderedDict as OD
import copy
import enum
import math
#import uproot3 as upSroot
import uproot as uproot
import hist
import matplotlib.pyplot as plt
import mplhep as hep
import scipy.stats as sts
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import LeaveOneOut


'''
References:
    Obtain Kernel Density Estimator from histogram, using scipy.stat.gaussian_kd1e https://stackoverflow.com/questions/53823349/how-can-you-create-a-kde-from-histogram-values-only
'''

#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20231122_MC_woTrg/2018/analyze_htoaa_stage1.root'
sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20231126_PNetHtoaa4bverQCD_SB/2018/plots_TransferFactor' 

sIpFiles = OD([
    ('ipFile1', '/eos/cms/store/user/ssawant/htoaa/analysis/20231126_PNetHtoaa4bverQCD_SB/2018/analyze_htoaa_stage1.root')
])

sOpFile = '%s/TransferFactor_SidebandToSR.root' % (sOpDir)

if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)
    
fIpFiles = OD()
for sIpFileName, sIpFileNameFull in sIpFiles.items():
    print(f"{sIpFileName = }, {sIpFileNameFull = }, ")
    fIpFiles[sIpFileName] = uproot.open(sIpFileNameFull)

fOpFile = uproot.recreate(sOpFile)

sIpFileName = 'ipFile1', sIpFileNameFull = '/eos/cms/store/user/ssawant/htoaa/analysis/20231126_PNetHtoaa4bverQCD_SB/2018/analyze_htoaa_stage1.root', 


TypeError: recreate() takes 1 positional argument but 2 were given

In [ ]:
for sHisto_ in fIpFiles['ipFile1']:
    if 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop' not in sHisto_: continue
    print(f"{sHisto_ = }")

sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_Presel_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP60_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP80_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_sel_leadingFatJetPt_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_sel_leadingFatJetEta_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_sel_JetID_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_sel_2018HEM1516Issue_central;1'
sHisto_ = 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_sel_leadingFatJetMSoftDrop_central;1'


In [ ]:
sXRange = "xAxisRange"; sYRange = "yAxisRange";
sXLabel = 'xAxisLabel'; sYLabel = 'yAxisLabel';
sXScale = 'xAxisScale';
sNRebin = "nRebin"
sHistosToOverlay = 'histosToOverlay'
sHistosToHadd = 'histosToHadd'
sIpFileNameNice = 'ipFileNameNice'
sHistName   = 'histogramName'
sNPseudoDataFactor = 'nPseudodataFactor'

printLevel = 0
#nPseudodataFactor = 50

cmsWorkStatus                  = 'Work in Progress'
era                            = '2018'
np.random.seed(1)

#colors_bkg_list = ['blue', 'orange', 'brown'] # ["#9b59b6", "#e74c3c", "#34495e", "#2ecc71"] #['lightcoral', 'burlywood', 'cyan', 'saddlebrown', 'slateblue', 'lightpink', 'darkkhaki', 'antiquewhite', 'limegreen', 'violet', 'firebrick', 'darkorchid', 'tan', 'olive', 'purple']

colors_bkg_list = [ 
    # ['color', <transperent>, '<fill pattern>']
    ['lightcoral',    0.9,  ''],
    ['cyan',          0.9,  '' ],
    ['burlywood',     0.9,  '' ], 
    ['saddlebrown',   0.9,  '' ],
    ['slateblue',     0.9,  '' ],
    ['lightpink',     0.9,  'xx' ],
    ['darkkhaki',     0.9,  '' ],
    ['antiquewhite',  0.9,  '//' ],
    ['limegreen',     0.9,  '' ],
    ['violet',        0.9,  '' ],
    ['firebrick',     0.9,  '' ],
    ['darkorchid',    0.9,  '' ],
    ['tan',           0.9,  '' ],
    ['olive',         0.9,  '' ],
    ['purple',        0.9,  ''],
]

colors_sig_list = [
    # ['color', <transperent>, '<fill pattern>', ]
    ['black',          0.9,  ''],
    ['blue',          0.9,  ''],
    ['red',           0.9,  ''],
    ['green',         0.9,  ''],
    ['magenta',       0.9,  ''],
    ['orange',        0.9,  ''],
]

histoNameShort_dict = OD([
    ('hLeadingFatJetMSoftDrop', {sXRange: [20, 300], sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetMass', {sXRange: [0, 300], sXLabel: 'Mass [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_mass', {sXRange: [0, 300], sXLabel: 'ParticleNet_mass [GeV]', sYLabel: 'Events',}),
     
    ('hLeadingFatJetParticleNet_massA_Hto4b_v0', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_v0 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_v1', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_v1 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_v2', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_v2 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_v3', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_v3 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_v4', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_v4 [GeV]', sYLabel: 'Events',}),
     
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v01', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v01 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v02', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v02 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v03', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v03 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v12', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v12 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v13', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v13 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v23', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v23 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v012', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v012 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v013', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v013 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v023', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v023 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v123', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v123 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massA_Hto4b_avg_v0123', {sXRange: [0, 300], sXLabel: 'ParticleNet_massA_Hto4b_avg_v0123 [GeV]', sYLabel: 'Events',}),
     
    ('hLeadingFatJetParticleNet_massH_Hto4b_v0', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v0 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massH_Hto4b_v00', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v00 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massH_Hto4b_v1', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v1 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massH_Hto4b_v2', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v2 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massH_Hto4b_v3', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v3 [GeV]', sYLabel: 'Events',}),
    ('hLeadingFatJetParticleNet_massH_Hto4b_v4', {sXRange: [0, 300], sXLabel: 'ParticleNet_massH_Hto4b_v4 [GeV]', sYLabel: 'Events',}),

    #('', {sXRange: [0, 300], sXLabel: ' [GeV]', sYLabel: 'Events',}),
    #('', {sXRange: [0, 300], sXLabel: ' [GeV]', sYLabel: 'Events',}),
    #('', {sXRange: [0, 300], sXLabel: ' [GeV]', sYLabel: 'Events',}),
    
    
])

#histoNameShort_dict = OD([
#    ('hLeadingFatJetMSoftDrop', {sXRange: [20, 300], sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',}),
#    ('hLeadingFatJetMass', {sXRange: [0, 300], sXLabel: 'Mass [GeV]', sYLabel: 'Events',}),
#])

sampleCategories_list = [
    'QCD', 
    'QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat'
]

histograms_dict = OD()
for histoNameShort, histo_dict_ in histoNameShort_dict.items():
    histoNameShort_1 = histoNameShort.replace('hLeadingFatJet', '')

    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},        
    ]

    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_0bCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},      
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},       
    ]
    
    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_1bCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},    
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},    
    ]
    
    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_2bCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},    
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
    ]
    
    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_3bCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},       
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},       
    ]
    
    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_4bCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
    ]
    
    histoNameNice_ = 'TransferFactor_SBWP80to40_to_SRWP40_QCD_5bAndMoreCat_%s' % (histoNameShort_1)
    histograms_dict[histoNameNice_] = copy.copy(histo_dict_)
    histograms_dict[histoNameNice_]['N'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        
    ]
    histograms_dict[histoNameNice_]['D'] = [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},        
    ]
    

    '''
    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD0bCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},    
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
        ],
    }

    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD1bCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},      
        ],
    }

    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD2bCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},      
        ],
    }

    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD3bCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},      
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
        ],
    }

    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD4bCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},   
        ],
    }

    histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD5bAndMoreCat'] = {
        sXRange: [20, 300],
        sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
        'N': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        
        ],
        'D': [
            {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},        
        ],
    }
    '''




'''
histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},        
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD0bCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},    
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD1bCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},      
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD2bCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},      
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD3bCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},      
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},     
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD4bCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},     
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},   
    ],
}

histograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD5bAndMoreCat'] = {
    sXRange: [20, 300],
    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',
    'N': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        
    ],
    'D': [
        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},        
    ],
}
'''

"\nhistograms_dict['TransferFactor_SBWP80to40_to_SRWP40_QCD'] = {\n    sXRange: [20, 300],\n    sXLabel: 'Msoftdrop [GeV]', sYLabel: 'Events',\n    'N': [\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_2bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_3bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_4bCat/hLeadingFatJetMSoftDrop_SRWP40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_5bAndMoreCat/hLeadingFatJetMSoftDrop_SRWP40_central'},        \n    ],\n    'D': [\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_0bCat/hLeadingFatJetMSoftDrop_SBWP80to40_central'},\n        {sIpFileNameNice: 'ipFile1', sHistName: 'evt/QCD_1bCat/

In [ ]:
def calChi2(h, f, var):
    chi2 = np.divide(np.square(h - f), var, where=var > 0, out=np.zeros(var.shape) )
    chi2 = np.sum(chi2)
    return chi2


In [ ]:
def getNonZeroMin(arr):
    min_ = 1e20
    #a_   = arr[np.nonzero(arr)]
    a_ = arr[ np.argwhere(arr > 0) ]
    if len(a_) > 0:
        min_ = np.min( a_ )
    return min_

In [ ]:
for plotNameNice in histograms_dict.keys():
    
    if printLevel >= 0:
        print(f"\n\nplotNameNice: {plotNameNice}")

    xAxisLabel = histograms_dict[plotNameNice][sXLabel] if sXLabel in list(histograms_dict[plotNameNice].keys()) else None
    yAxisLabel = histograms_dict[plotNameNice][sYLabel] if sYLabel in list(histograms_dict[plotNameNice].keys()) else None
    xAxisRange = histograms_dict[plotNameNice][sXRange] if sXRange in list(histograms_dict[plotNameNice].keys()) else None
    yAxisRange = histograms_dict[plotNameNice][sYRange] if sYRange in list(histograms_dict[plotNameNice].keys()) else None 
    xAxisScale = histograms_dict[plotNameNice][sXScale] if sXScale in list(histograms_dict[plotNameNice].keys()) else ""
        
    nRebins = histograms_dict[plotNameNice][sNRebin] if sNRebin in list(histograms_dict[plotNameNice].keys()) else 1
    
    hTransferFactor_dict = {}
    for sHistoType in ['N', 'D']:     # ['N', 'D']   

        fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(8,10), sharex='col', gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})
        
        h_added = None
        for iHistoDetails_dict in histograms_dict[plotNameNice][sHistoType]: # histograms to hadd
            sIpFileName_    = iHistoDetails_dict[sIpFileNameNice]            
            sHistogramName_ = iHistoDetails_dict[sHistName]
            fIp_            = fIpFiles[sIpFileName_]
            h_              = fIp_[sHistogramName_].to_hist()

            if not isinstance(h_added, type(h_)):
                h_added = h_
            else:
                h_added = h_added + h_

            if printLevel >= 11:
                print(f"\n\n{plotNameNice = }, {sHistoType = }")
                print(f"{h_.values() = }")
                print(f"{h_.variances() = }")
                print(f"\n{h_added.values() = }")
                print(f"{h_added.variances() = }")
            
        hep.histplot(
            h_added.values(), 
            bins=h_added.axes[0].edges, 
            ax=ax[0], 
            histtype='errorbar', 
            #stack=True, 
            label='Histogram', 
            color=colors_sig_list[0][0],
            marker='o',
            markerfacecolor=colors_sig_list[0][0],
            markersize=3
            )

        
        xEdges   = h_added.axes[0].edges
        xCenters = h_added.axes[0].centers
        nXbins   = xCenters.shape[0]
        xCenters_finer = np.linspace(xEdges.min(), xEdges.max(), num=nXbins*10)
        
        hRef = h_added / ( np.diff(xEdges)  * h_added.values().sum() )
        sf_KDE = h_added.values().sum() / hRef.values().sum()


        #nPseudodata_toUse = int(nXbins * nPseudodataFactor) 
        nPseudodata_toUse = int(h_added.values().sum() / 4.)
        if nPseudodata_toUse < 20:
            nPseudodata_toUse = 20

        if printLevel >=12:
            print(f"{h_added.axes[0].edges = }")
            print(f"{np.diff(h_added.axes[0].edges) = }")
            print(f"{h_added.values() = }, {h_added.values().sum()}")
        if printLevel >=11:
            print(f"{h_added.values().sum() = }, {hRef.values().sum() = }")
            print(f"{hRef.values() = }, {hRef.values().sum() = }")
        if printLevel >=0:
            print(f"{h_added.values().sum() = }, {nPseudodata_toUse = }")


        #pseudodata = np.random.choice(h_added.axes[0].centers, size=1e6, p=(hRef/hRef.values().sum()).values())
        pseudodata = np.random.choice(h_added.axes[0].centers, size=nPseudodata_toUse, p=hRef.values())        
        pseudoKDE   = sts.gaussian_kde(pseudodata)
        #pseudoKDE_1 = sts.gaussian_kde(pseudodata, bw_method='silverman')
        #pseudoKDE_2 = KernelDensity(kernel='gaussian', bandwidth=1.0).fit(pseudodata[:, None])
        #bandwidths = 10 ** np.linspace(-1, 1, 10)
        #grid_ = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth':bandwidths}, cv=LeaveOneOut())
        #grid_.fit(pseudodata[:, None])
        #print(f"{grid_ = }, {grid_.best_params_ = }, {grid_.best_params_['bandwidth'] = }")
        #print(f"{grid_.best_estimator_.score_samples(xCenters_finer[:, None]) = }")
        
        hKDE = hist.Hist(hist.axis.Regular(nXbins,xEdges[0],xEdges[-1]), storage=hist.storage.Double())
        hKDE[:] = pseudoKDE.pdf(xCenters) * sf_KDE
        #hKDE.variances()[:] = h_added.variances()
        #chi2 = np.divide(np.square(h_added.values() - hKDE.values()), h_added.variances(), where=h_added.variances() > 0, out=np.zeros(h_added.variances().shape) )
        #chi2 = np.sum(chi2)
        chi2 = calChi2(h_added.values(), hKDE.values(), h_added.variances())
        
        hTransferFactor_dict[sHistoType] = copy.copy(hKDE)

        if printLevel >= 12:
            print(f"{xCenters_finer = } ")
            print(f"{pseudoKDE.pdf(xCenters_finer) =  }")
            print(f"{h_added.values().sum() = }, {hRef.values().sum() = }, {sf_KDE = }")
            print(f"{pseudoKDE.pdf(xCenters_finer) * sf_KDE =  }")

        if printLevel >= 12:
            print(f"{hKDE.values() = }")
            print(f"{hKDE.variances() = }")
            print(f"{chi2 = }, {chi2 / nXbins =}")        
        
        #print(f"{pseudoKDE.pdf(h_added.axes[0].centers) = }")
        '''
        hep.histplot(
            pseudoKDE.pdf(xCenters_finer) * h_added.values().sum() / hRef.values().sum(), 
            bins=h_added.axes[0].edges, 
            ax=ax[0], 
            #histtype='errorbar', 
            #stack=True, 
            #label=sStack_list, 
            #color=colors_toUse,
            #alpha=alpha_toUse,
            #hatch=hatch_toUse,
            )
        '''
        ax[0].plot(xCenters_finer, pseudoKDE  .pdf(xCenters_finer) * sf_KDE, '-', c='C3', lw=4, label='Pseudo-KDE(bw=Scott\'s Rule)  (nSample = %d) (chi2/ndf = %g)' % (nPseudodata_toUse, chi2/nXbins ))
        #chi2_ = calChi2(h_added.values(), pseudoKDE_1.pdf(xCenters) * sf_KDE, h_added.variances())
        #ax[0].plot(xCenters_finer, pseudoKDE_1.pdf(xCenters_finer) * sf_KDE, '-', c='C4', lw=4, label='Pseudo-KDE(bw=Silverman\'s Rule) (nSample = %d) (chi2/ndf = %g)' % (nPseudodata_toUse, chi2_/nXbins ))
        #ax[0].plot(xCenters_finer, np.exp(pseudoKDE_2.score_samples(xCenters_finer[:, None])) * sf_KDE, '-', c='C5', lw=4, label='Pseudo-KDE(bw=Scott) (nSample = %d) (chi2/ndf = %g)' % (nPseudodata_toUse, chi2/nXbins ))
        #chi2_ = calChi2(h_added.values(), np.exp(grid_.best_estimator_.score_samples(xCenters[:, None])) * sf_KDE, h_added.variances())
        #ax[0].plot(xCenters_finer, np.exp(grid_.best_estimator_.score_samples(xCenters_finer[:, None])) * sf_KDE, '-', c='C5', lw=4, label='Pseudo-KDE(bw=%.3f) (nSample = %d) (chi2/ndf = %g)' % (grid_.best_params_['bandwidth'], nPseudodata_toUse, chi2_/nXbins ))
        


            
        
        # Ratio plot ---------------------------------------------------------    
                   
        ratio_values = np.divide(h_added.values(), hKDE.values(), where=hKDE.values()!=0, out=np.ones(hKDE.values().shape))
        ratio_error  = np.sqrt(h_added.variances())           
        ratio_error  = np.divide(ratio_error, hKDE.values(), where=hKDE.values()!=0, out=np.zeros(hKDE.values().shape))
        #ratio_syst   = np.sqrt(hBkgTot_variance)
        #ratio_syst   = np.divide(ratio_syst, hBkgTot_values, where=hBkgTot_values!=0, out=np.zeros(hKDE.values().shape))    
        
        #hRatio = h_added / hKDE

        hep.histplot(
            ratio_values, 
            bins=h_added.axes[0].edges, 
            ax=ax[1], 
            yerr=ratio_error, 
            histtype='errorbar', 
            #label='Data'
            color=colors_sig_list[0][0],
            marker='o',
            markerfacecolor=colors_sig_list[0][0],
            markersize=3
            )

        # Upper plot cosmetics ---------
        if xAxisRange: ax[0].set_xlim(xAxisRange[0], xAxisRange[1])
        #print(f"\nAt the end {yAxisRange_cal = }")
        if yAxisRange: ax[0].set_ylim(yAxisRange[0], yAxisRange[1])     
        if xAxisLabel: ax[0].set_xlabel(xAxisLabel)
        if yAxisLabel: ax[0].set_ylabel(yAxisLabel)   
        ax[0].grid()  

        if xAxisLabel: ax[1].set_xlabel(xAxisLabel)   

        ax[0].legend(title='%s %s'%(plotNameNice, sHistoType), fontsize=12)

        # Ratio plot cosmetics ---------
        if xAxisRange: ax[1].set_xlim(xAxisRange[0], xAxisRange[1])
        ax[1].set_ylim(0, 2)
        if xAxisLabel: ax[1].set_xlabel(xAxisLabel)
        ax[1].set_ylabel('Histogram / KDE')
        
        ax[1].axhline(y=1, linestyle='--')
        ax[1].grid()        

        isData = False
        fontsize_toUse = 18 if isData else 15
        hep.cms.label(ax=ax[0], data=isData, year=era, label=cmsWorkStatus, fontsize=fontsize_toUse)

        '''
        selectionTag = "WP40"
        ax[0].text(0.8, 0.8, selectionTag,
            fontsize=12, fontstyle='italic',
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax[0].transAxes
            )
        '''

        fig.savefig('%s/%s_%s.png' % (sOpDir,plotNameNice,sHistoType), transparent=False, dpi=80, bbox_inches="tight")

        plt.close(fig)

    
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8,10))
    hTransferFactor = copy.copy(hTransferFactor_dict['N'])
    transferFactor_ = np.divide(hTransferFactor_dict['N'].values(), hTransferFactor_dict['D'].values(), where=hTransferFactor_dict['D'].values()>0, out=np.ones(hTransferFactor_dict['N'].values().shape) )
    if xAxisRange:
        mask_xAxisRange = np.logical_and( hTransferFactor.axes[0].centers > xAxisRange[0], hTransferFactor.axes[0].centers < xAxisRange[1] )
        transferFactor_ = np.where(
            mask_xAxisRange,
            transferFactor_,
            np.ones(hTransferFactor_dict['N'].values().shape) 
            )
        yMinRatio = getNonZeroMin(transferFactor_[mask_xAxisRange])
        yMaxRatio = np.max(transferFactor_[mask_xAxisRange])
    hTransferFactor[:] = transferFactor_

    if printLevel >= 1:
        print(f"{hTransferFactor.values() = }")

    hep.histplot(
        hTransferFactor.values(), 
        bins=hTransferFactor.axes[0].edges, 
        ax=ax, 
        #yerr=ratio_error, 
        histtype='step', 
        label=plotNameNice,
        color=colors_sig_list[0][0],
        #marker='o',
        #markerfacecolor=colors_sig_list[0][0],
        #markersize=3
        )
    
    ax.legend()

    if xAxisRange: ax.set_xlim(xAxisRange[0], xAxisRange[1])
    if xAxisRange: ax.set_ylim(yMinRatio*0.8, yMaxRatio*1.2)
    #print(f"\nAt the end {yAxisRange_cal = }")
    #if yAxisRange: ax.set_ylim(yAxisRange[0], yAxisRange[1])     
    if xAxisLabel: ax.set_xlabel(xAxisLabel)
    if yAxisLabel: ax.set_ylabel('Transfer Factor') 
    ax.grid()

    fOpFile['%s' % (plotNameNice)] = hTransferFactor

    fig.savefig('%s/%s.png' % (sOpDir,plotNameNice), transparent=False, dpi=80, bbox_inches="tight")

    plt.close(fig)


fOpFile.close()



plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_MSoftDrop
h_added.values().sum() = 6404.709988334146, 1601
h_added.values().sum() = 53827.16357408302, 13456


plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_0bCat_MSoftDrop
h_added.values().sum() = 81.65512158168478, 20
h_added.values().sum() = 1733.3501045058256, 433


plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_1bCat_MSoftDrop
h_added.values().sum() = 14.072105731958747, 20
h_added.values().sum() = 790.1970634030203, 197


plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_2bCat_MSoftDrop
h_added.values().sum() = 1904.6702413702385, 476
h_added.values().sum() = 30397.91148830275, 7599


plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_3bCat_MSoftDrop
h_added.values().sum() = 803.5344152599381, 200
h_added.values().sum() = 7256.730713402645, 1814


plotNameNice: TransferFactor_SBWP80to40_to_SRWP40_QCD_4bCat_MSoftDrop
h_added.values().sum() = 3521.6430599159107, 880
h_added.values().sum() = 13437.04204718